In [1]:
import pandas as pd
import geopandas as gpd

from shared_utils import rt_utils, catalog_utils

from calitp_data_analysis import get_fs, geography_utils
from segment_speed_utils import helpers, time_series_utils, gtfs_schedule_wrangling, corridor_analysis
from segment_speed_utils.project_vars import SCHED_GCS, SEGMENT_GCS, GTFS_DATA_DICT, analysis_date

ModuleNotFoundError: No module named 'shared_utils'

# develop and test some basic tools for corridor analysis

In [ ]:
catalog = catalog_utils.get_catalog('gtfs_analytics_data')

In [ ]:
catalog.speedmap_segments

In [ ]:
analysis_date

In [ ]:
# path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.stage4}_{analysis_date}.parquet'

In [ ]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet'

In [ ]:
path

In [ ]:
detail = gpd.read_parquet(path)

In [ ]:
detail.head(3)

## need trip-level (pre-aggregation) gdf to properly calculate metrics

In [ ]:
st4 = corridor_analysis.import_trip_speeds(analysis_date)

## corridor specification

In [ ]:
orgs = detail.drop_duplicates(subset=['organization_source_record_id', 'organization_name'])[['organization_source_record_id', 'organization_name']]

In [ ]:
orgs[orgs.organization_name.str.contains('San Fr')]

## define corridor

In [ ]:
# shape_id = '4953'
# start_seg_id = '18088-18089-1'
# end_seg_id = '16800-16806-1'

sf = 'rechaapWbeffO33OX'
shape_id = '800'
start_seg_id = '14970-17900-2'
end_seg_id = '16357-16358-1'

In [ ]:
gdf = corridor_analysis.corridor_from_segments(speed_segments_gdf=detail, organization_source_record_id=sf, shape_id=shape_id,
                      start_seg_id=start_seg_id, end_seg_id=end_seg_id)

In [ ]:
# gdf.explore()

## Corridor Measurements

Previous logic:

For each trip, get from the last stop before entering corridor to the first stop after exiting corridor. This was done on stop_sequence

Now,

* first sjoin with aggregated data (has geom). Sjoining on segments is equivalent to previous methodology, since it will yield the last stop before entry to the first stop after exiting...
* avoid doing scheduled delay metric for now...
* 

In [ ]:
corridor_trips = corridor_analysis.find_corridor_data(detail, gdf, st4)

In [ ]:
corridor_results = corridor_analysis.analyze_corridor_trips(corridor_trips)

In [ ]:
corridor_results

In [ ]:
corridor_results.groupby('route_short_name')['corridor_speed_mph'].describe()

In [ ]:
routes = ['8AX', '8BX', '8']
rt8 = corridor_results.query('route_short_name in @routes')

In [ ]:
corridor_analysis.analyze_corridor_improvements(rt8, trip_seconds_saved=30)